#Sistema de recomendação de animes

---
Montar um sistema de recomendação de animes com os dados do MyAnimeList.
Os animes recomendados são baseados em:

* Avaliação de cada usuário
* Gênero do anime selecionado


## Manipulação dos dados

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_anime = pd.read_csv('anime.csv')
df_notas = pd.read_csv('rating.csv')

In [4]:
display(df_anime.head())
df_notas.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


,user_id,anime_id,rating
0,1,20.0,-1.0
1,1,24.0,-1.0
2,1,79.0,-1.0
3,1,226.0,-1.0
4,1,241.0,-1.0


In [5]:
display(df_anime.info())
print('=='*40)
display(df_notas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7292434 entries, 0 to 7292433
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   int64  
 1   anime_id  float64
 2   rating    float64
dtypes: float64(2), int64(1)
memory usage: 166.9 MB


None

Agora uniremos os dois dataframes em um só, pela coluna 'anime_id', excluindo o 'rating' do df_anime (porque o que nos interessa aqui é a nota dada pelo usuário, o 'user_id')

In [6]:
df = pd.merge(df_notas, df_anime.drop('rating', axis=1),on='anime_id' )
print(df.shape)
df.head()

(7292423, 8)


,user_id,anime_id,rating,name,genre,type,episodes,members
0,1,20.0,-1.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
1,3,20.0,8.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
2,5,20.0,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
3,6,20.0,-1.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
4,10,20.0,-1.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297


Fazer um agrupamento dos animes pelo nome, e pegar a contagem das avaliações. Com isso, teremos os animes com mais avaliações (não as maiores avaliações, mas aqueles com mais QUANTIDADE DE VOTOS, sejam eles altos ou baixos).

Depois de pegar esses animes, apenas os colocaremos em ordem descrescente, colocando depois do código um:

```
.sort_values(ascending=False)
```



In [35]:
df.groupby('name')['rating'].count().sort_values(ascending=False).head(600)

name
Death Note                                                                                            36400
Sword Art Online                                                                                      27753
Shingeki no Kyojin                                                                                    26739
Code Geass: Hangyaku no Lelouch                                                                       25826
Elfen Lied                                                                                            25619
                                                                                                      ...  
Ryuugajou Nanana no Maizoukin (TV)                                                                     3283
Seirei Tsukai no Blade Dance                                                                           3279
Sengoku Basara                                                                                         3265
Dragon Ball Z Movie 12:

Depois disso, criaremos um dataframe para a média das notas:
Criar o dataframe
Criar uma coluna com os números de avaliações (as quantidades)
Criar uma coluna com as médias de avaliações

In [8]:
notas = pd.DataFrame(df.groupby('name')['rating'].mean())
notas['numero de avaliações'] = pd.DataFrame(df.groupby('name')['rating'].count())
notas['avaliação média'] = pd.DataFrame(df.groupby('name')['rating'].mean().round(2))
notas

,rating,numero de avaliações,avaliação média
name,,,
&quot;0&quot;,2.800000,25,2.80
"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",0.000000,2,0.00
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,5.822581,744,5.82
&quot;Bungaku Shoujo&quot; Memoire,6.165580,767,6.17
&quot;Bungaku Shoujo&quot; Movie,6.475296,1437,6.48
...,...,...,...
xxxHOLiC Kei,6.743661,3234,6.74
xxxHOLiC Movie: Manatsu no Yoru no Yume,6.340979,2267,6.34
xxxHOLiC Rou,6.445607,1434,6.45


Agora criar um novo dataframe para os generos, usando o nome e o genero dos animes no df_anime original,
Depois colocar o nome do anime como indice do dataframe do genero

In [9]:
genero = pd.DataFrame(data=df_anime[['name','genre']])
genero.set_index('name', inplace=True)
genero

,genre
name,
Kimi no Na wa.,"Drama, Romance, School, Supernatural"
Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
Steins;Gate,"Sci-Fi, Thriller"
Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."
...,...
Toushindai My Lover: Minami tai Mecha-Minami,Hentai
Under World,Hentai
Violence Gekiga David no Hoshi,Hentai


## Sistema de recomendação

---
Agora que já limpamos e organizamos os dados, é hora de criar o sistema de recomendação


In [36]:
#O primeiro passo é criar uma função para verificar o genero do anime, essa função será usado dentro do nosso sistema
def verificar_genero(lista_genero, string): #Definir a função, as variaveis são a lista de genero e uma string
    if any(x in string for x in lista_genero): #Se existir um x nessa string que está dentro da lista de generos
        return True #Vai retornar True
    else: #Se não
        return False #Vai retornar False

#Agora é o sistema de recomendação


#Definir a função para recomendar o anime, usando como variável o nome do anime e o número n de animes que será recomendado
def recomendar_anime(nome_do_anime, n):

    #Localizar o anime escolhido dentro do dataframe genero que criamos lá em cima, depois pegar os valores contidos nessa série e splitar
    genero_anime = genero.loc[nome_do_anime].values[0].split(', ')
    
    #Colocar o nome dos animes do mesmo gênero que o escolhido e colocar numa lista
    cols = df_anime[df_anime['genre'].apply(lambda x: verificar_genero(genero_anime, str(x)))]['name'].tolist()

    #Pegar o nome dos animes que foram selecionados e fazer uma pivot_table com o df (que foi unido lá em cima) e a nota que cada usuário deu pra esse anime
    matriz_de_animes = df[df['name'].isin(cols)].pivot_table(index='user_id', columns='name', values='rating')

    #Verificar a nota de cada usuário para aquele anime selecionado
    anime_nota = matriz_de_animes[nome_do_anime]

    #Fazer a correlação entre os nomes dos animes e suas notas
    anime_parecido = matriz_de_animes.corrwith(anime_nota)

    #Criar um dataframe com essa correlação e colocar numa coluna 'correlação'
    anime_correlacionado = pd.DataFrame(anime_parecido, columns=['correlação'])

    #Pegar o dataframe de notas que criamos lá em cima e selecionar as colunas de numero de avaliações e avaliação média
    anime_correlacionado = anime_correlacionado.join(notas[['numero de avaliações', 'avaliação média']])

    #Excluir os dados nulos
    anime_correlacionado.dropna(inplace=True)

    #Selecionar apenas os animes com número de avaliação maior que 5 mil, colocá-los em ordem descrescente
    animes_recomendados = anime_correlacionado[anime_correlacionado['numero de avaliações'] > 3000].sort_values('correlação', ascending=False)

    #Criar nosso dataframe final, que vai ser a tabela dos animes recomendados
    animes_recomendados= animes_recomendados.rename_axis('Animes recomendados')
    print(f'Anime escolhido: {nome_do_anime}')
    return animes_recomendados.head(n+1)

E aqui está nosso sistema de recomendação de animes. Agora vamos testar!!

Escolherei os 5 animes com mais avaliações

In [11]:
df.groupby('name')['rating'].count().sort_values(ascending=False).head()

name
Death Note                         36400
Sword Art Online                   27753
Shingeki no Kyojin                 26739
Code Geass: Hangyaku no Lelouch    25826
Elfen Lied                         25619
Name: rating, dtype: int64

In [37]:
recomendar_anime('Death Note', 5)

Anime escolhido: Death Note


,correlação,numero de avaliações,avaliação média
Animes recomendados,,,
Death Note,1.000000,36400,7.56
Claymore,0.710327,11518,6.84
Higurashi no Naku Koro ni Kai,0.707356,8678,7.11
Blood+,0.706640,7109,6.48
Darker than Black: Kuro no Keiyakusha,0.703644,14608,7.02
Beelzebub,0.701573,5433,6.60


In [38]:
recomendar_anime('Sword Art Online',5)

Anime escolhido: Sword Art Online


,correlação,numero de avaliações,avaliação média
Animes recomendados,,,
Sword Art Online,1.000000,27753,6.88
Sword Art Online II,0.753359,12211,6.18
Koi to Senkyo to Chocolate,0.735017,3050,5.99
Oda Nobuna no Yabou,0.731385,4644,6.38
Btooom!,0.729014,11061,6.46
Zero no Tsukaima F,0.728047,6723,6.20


In [14]:
recomendar_anime('Shingeki no Kyojin',5)

Anime escolhido: Shingeki no Kyojin


,correlação,numero de avaliações,avaliação média
Animes recomendados,,,
Shingeki no Kyojin,1.000000,26739,7.32
Suisei no Gargantia,0.758896,6001,6.36
Kami nomi zo Shiru Sekai: Megami-hen,0.748214,5287,6.82
Blood Lad,0.740993,7825,6.09
Maoyuu Maou Yuusha,0.733103,5311,6.03
Danganronpa: Kibou no Gakuen to Zetsubou no Koukousei The Animation,0.725263,9238,5.99


In [15]:
recomendar_anime('Code Geass: Hangyaku no Lelouch',5)

Anime escolhido: Code Geass: Hangyaku no Lelouch


,correlação,numero de avaliações,avaliação média
Animes recomendados,,,
Code Geass: Hangyaku no Lelouch,1.000000,25826,7.66
Code Geass: Hangyaku no Lelouch R2,0.888114,22583,7.77
Darker than Black: Kuro no Keiyakusha,0.706963,14608,7.02
Beelzebub,0.706000,5433,6.60
Clannad,0.699624,20219,7.23
Claymore,0.693905,11518,6.84


In [16]:
recomendar_anime('Elfen Lied',5)

Anime escolhido: Elfen Lied


,correlação,numero de avaliações,avaliação média
Animes recomendados,,,
Elfen Lied,1.000000,25619,6.77
Elfen Lied Special,0.754194,7003,5.67
Chobits,0.727232,11762,6.26
Shakugan no Shana,0.725577,11277,6.46
Fate/stay night,0.719264,14041,6.48
Rosario to Vampire Capu2,0.717943,9568,5.90


E, por último, o meu anime preferido: Samurai Champloo

In [21]:
recomendar_anime('Samurai Champloo',5)

Anime escolhido: Samurai Champloo


,correlação,numero de avaliações,avaliação média
Animes recomendados,,,
Samurai Champloo,1.000000,13205,7.25
Ghost in the Shell: Stand Alone Complex,0.751186,5988,7.05
Sayonara Zetsubou Sensei,0.749126,5810,6.75
Full Metal Panic! The Second Raid,0.748453,7708,6.66
Rurouni Kenshin: Meiji Kenkaku Romantan,0.747438,7462,6.92
Yuu☆Yuu☆Hakusho,0.738948,6356,6.95
